In [2]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup


In [3]:
df = pd.read_csv('../data-analysis-pipeline/Input/results.csv')
df

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
41238,2019-10-20,Lesotho,Zimbabwe,0,0,African Nations Championship qualifying,Lesotho,Lesotho,False
41239,2019-10-20,Guinea,Senegal,1,0,African Nations Championship qualifying,Conakry,Guinea,False
41240,2019-10-20,Mali,Mauritania,2,0,African Nations Championship qualifying,Mali,Mali,False
41241,2019-10-20,Burkina Faso,Ghana,0,0,African Nations Championship qualifying,Burkina Faso,Burkina Faso,False


In [32]:
df['date'] = pd.to_datetime(df['date'])


In [33]:
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [34]:
df = df.loc[(df['year'] > 1989)]


In [35]:
coleliminar = ['date','city','country', 'neutral', 'day', 'month', 'year']
df = df.drop(coleliminar, axis=1)

In [36]:
def tournament(e):
    listtourn = ['CONCACAF Nations League', 'Gold Cup qualification', 'Oceania Nations Cup', 'Friendly', 'African Cup of Nations qualification', 'UEFA Euro', 'African Nations Championship qualifying', 'UEFA Nations League', 'African Cup of Nations', 'Copa América qualification', 'Gold Cup', 'FIFA World Cup', 'Oceania Nations Cup qualification', 'CONCACAF Nations League qualifying', 'Confederations Cup', 'UEFA Euro qualification', 'Copa América', 'FIFA World Cup qualification']
    if e in listtourn:
        return e
    else:
        return 'Unofficial tournament'
#isin function mirar
df['tournament'] = df['tournament'].apply(tournament)    

In [37]:
df = df.loc[(df['tournament']!= 'No vale')]

In [38]:
def points(row1,row2):
    row = 0
    if row1 > row2:
        row +=3
    elif row1 == row2:
        row +=1
    elif row1 < row2:
        row = 0
    return row


In [39]:
for index, row in df.iterrows():
    df.loc[index,"home_points"] = points(row['home_score'], row['away_score'])


In [40]:
for index, row in df.iterrows():
    df.loc[index, "away_points"] = points(row['away_score'], row['home_score'])

In [41]:
df= df[['home_team','away_team','home_points', 'away_points']]

In [42]:
df[['home_team', 'away_team']] = df[['home_team', 'away_team']].apply(lambda x: x.str.upper())


In [43]:
df

,home_team,away_team,home_points,away_points
15872,ALGERIA,MALI,3.0,0.0
15873,ALGERIA,CAMEROON,3.0,0.0
15874,GREECE,BELGIUM,3.0,0.0
15875,MEXICO,ARGENTINA,3.0,0.0
15876,MALAWI,TANZANIA,1.0,1.0
...,...,...,...,...
41238,LESOTHO,ZIMBABWE,1.0,1.0
41239,GUINEA,SENEGAL,3.0,0.0
41240,MALI,MAURITANIA,3.0,0.0
41241,BURKINA FASO,GHANA,1.0,1.0


In [44]:
home = df.groupby('home_team', as_index=False).agg({'home_points': 'mean'})

In [45]:
home = home.rename(columns = {'home_team': 'Country'})

In [46]:
away = df.groupby('away_team', as_index=False).agg({'away_points':'mean'})

In [47]:
away = away.rename(columns = {'away_team':'Country'})

In [48]:
definitive = pd.merge(home, away, on='Country')

In [49]:
definitive['MPP'] = definitive['home_points'] + definitive['away_points']

In [50]:
definitive = definitive[['Country','MPP']]

In [51]:
lista = ['ANDALUSIA', 'BASQUE COUNTRY', 'KERNOW', 'ARTSAKH', 'YORKSHIRE', 'YUGOSLAVIA', 'ABKHAZIA', 'JERSEY', 'CZECHOSLOVAKIA', 'MICRONESIA']

In [52]:
definitive = definitive.loc[(~definitive['Country'].isin(lista))]


In [53]:
definitive = definitive.sort_values(by='MPP', ascending = False)

In [54]:
definitive = definitive.reset_index()

In [55]:
definitive = definitive[['Country', 'MPP']]

In [56]:
definitive['Position'] = definitive.index + 1

In [57]:
definitive = definitive[['Position', 'Country', 'MPP']]

In [58]:
definitive

,Position,Country,MPP
0,1,GERMAN DR,5.000000
1,2,SPAIN,4.399058
2,3,BRAZIL,4.295744
3,4,FRANCE,4.132602
4,5,GERMANY,4.083409
...,...,...,...
228,229,ANGUILLA,0.443850
229,230,EAST TIMOR,0.428571
230,231,ANDORRA,0.422602
231,232,SAN MARINO,0.086728


In [517]:
position=list(definitive.loc[(definitive['Country'] == 'BELGIUM'), 'Position'])

In [524]:
position = position[0]

In [525]:
type(position)

int

In [526]:
def database(country):
    df = pd.read_csv('../data-analysis-pipeline/Input/results.csv')
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df = df.loc[(df['year'] > 1990)]
    coleliminar = ['date', 'city', 'country', 'neutral', 'day', 'month', 'year']
    df = df.drop(coleliminar, axis=1)

    def tournament(e):
        listtourn = ['CONCACAF Nations League', 'Gold Cup qualification', 'Oceania Nations Cup', 'Friendly', 'African Cup of Nations qualification', 'UEFA Euro', 'African Nations Championship qualifying', 'UEFA Nations League', 'African Cup of Nations', 'Copa América qualification', 'Gold Cup', 'FIFA World Cup', 'Oceania Nations Cup qualification', 'CONCACAF Nations League qualifying', 'Confederations Cup', 'UEFA Euro qualification', 'Copa América', 'FIFA World Cup qualification']
        if e in listtourn:
            return e
        else:
            return 'No vale'
    df['tournament'] = df['tournament'].apply(tournament)
    df = df.loc[(df['tournament'] != 'No vale')]

    def points(value1, value2):
        value = 0
        if value1 > value2:
            value += 3
        elif value1 == value2:
            value += 1
        elif value1 < value2:
            value == 0
        return value
    
    for index, row in df.iterrows():
        df.loc[index, 'home_points'] = points(row['home_score'], row['away_score'])

    for index, row in df.iterrows():
        df.loc[index, 'away_points'] = points(row['away_score'], row['home_score'])
    
    df = df[['home_team', 'away_team', 'home_points', 'away_points']]
    df[['home_team']] = df[['home_team']].apply(lambda x: x.str.upper())
    df[['away_team']] = df[['away_team']].apply(lambda x: x.str.upper())
    home = df.groupby('home_team', as_index=False).agg({'home_points': 'mean'})
    home = home.rename(columns = {'home_team': 'Country'})
    away = df.groupby('away_team', as_index=False).agg({'away_points':'mean'})
    away = away.rename(columns = {'away_team':'Country'})
    definitive = pd.merge(home, away, on='Country')
    definitive['MPP'] = definitive['home_points'] + definitive['away_points']
    definitive = definitive[['Country','MPP']]
    lista = ['ANDALUSIA', 'BASQUE COUNTRY', 'KERNOW', 'ARTSAKH', 'YORKSHIRE', 'YUGOSLAVIA', 'ABKHAZIA', 'JERSEY', 'CZECHOSLOVAKIA', 'MICRONESIA']
    definitive = definitive.loc[(~definitive['Country'].isin(lista))]
    definitive = definitive.sort_values(by='MPP', ascending = False)
    definitive = definitive.reset_index()
    definitive = definitive[['Country', 'MPP']]
    definitive['Position'] = definitive.index + 1
    definitive = definitive[['Position', 'Country', 'MPP']]
    position = list(definitive.loc[(definitive['Country'] == country), 'Position'])
    position = position[0]
    return "{} is the {} best country in football for the last 30 years.".format(country, position)

In [527]:
database('BURKINA FASO')

'BURKINA FASO is the 109 best country in football for the last 30 years.'

In [ ]:
def database1(country):
    df = pd.read_csv('../data-analysis-pipeline/Input/results.csv')
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df = df.loc[(df['year'] > 1990)]
    coleliminar = ['date', 'city', 'country', 'neutral', 'day', 'month', 'year']
    df = df.drop(coleliminar, axis=1)

    def tournament(e):
        listtourn = ['CONCACAF Nations League', 'Gold Cup qualification', 'Oceania Nations Cup', 'Friendly', 'African Cup of Nations qualification', 'UEFA Euro', 'African Nations Championship qualifying', 'UEFA Nations League', 'African Cup of Nations', 'Copa América qualification', 'Gold Cup', 'FIFA World Cup', 'Oceania Nations Cup qualification', 'CONCACAF Nations League qualifying', 'Confederations Cup', 'UEFA Euro qualification', 'Copa América', 'FIFA World Cup qualification']
        if e in listtourn:
            return e
        else:
            return 'No vale'
    df['tournament'] = df['tournament'].apply(tournament)
    df = df.loc[(df['tournament'] != 'No vale')]

    def points(value1, value2):
        value = 0
        if value1 > value2:
            value += 3
        elif value1 == value2:
            value += 1
        elif value1 < value2:
            value == 0
        return value
    
    for index, row in df.iterrows():
        df.loc[index, 'home_points'] = points(row['home_score'], row['away_score'])

    for index, row in df.iterrows():
        df.loc[index, 'away_points'] = points(row['away_score'], row['home_score'])
    
    df = df[['home_team', 'away_team', 'home_points', 'away_points']]
    df[['home_team']] = df[['home_team']].apply(lambda x: x.str.upper())
    df[['away_team']] = df[['away_team']].apply(lambda x: x.str.upper())
    home = df.groupby('home_team', as_index=False).agg({'home_points': 'mean'})
    home = home.rename(columns = {'home_team': 'Country'})
    away = df.groupby('away_team', as_index=False).agg({'away_points':'mean'})
    away = away.rename(columns = {'away_team':'Country'})
    definitive = pd.merge(home, away, on='Country')
    definitive['MPP'] = definitive['home_points'] + definitive['away_points']
    definitive = definitive[['Country','MPP']]
    lista = ['ANDALUSIA', 'BASQUE COUNTRY', 'KERNOW', 'ARTSAKH', 'YORKSHIRE', 'YUGOSLAVIA', 'ABKHAZIA', 'JERSEY', 'CZECHOSLOVAKIA', 'MICRONESIA']
    definitive = definitive.loc[(~definitive['Country'].isin(lista))]
    definitive = definitive.sort_values(by='MPP', ascending = False)
    definitive = definitive.reset_index()
    definitive = definitive[['Country', 'MPP']]
    definitive['Position'] = definitive.index + 1
    definitive = definitive[['Position', 'Country', 'MPP']]
    if country in definitive['Country']:
        position = definitive.loc[(definitive['Country'] == country), 'Position'].iloc[0]
    return "{} is the {} best country in football for the last 30 years.".format(country, position)

#WEB SCRAPPING

In [126]:
url = "https://www.fifa.com/fifa-world-ranking/ranking-table/men/"
res = requests.get(url)
html = res.text

In [134]:

def scrapping(url):
    url = "https://www.fifa.com/fifa-world-ranking/ranking-table/men/"
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    fifa = soup.select('tbody')[0].find_all('tr')
    return fifa

In [135]:
scrapping(url)

[<tr data-team-id="43935">
 <td class="fi-table__td fi-table__rank"><span class="text">1</span></td>
 <td class="fi-table__td fi-table__teamname"><span class="text"> <a class="fi-t__link" href="/fifa-world-ranking/associations/association/BEL/men/">
 <div class="fi-t fi-i--4" data-team-id="43935">
 <div class="fi-t__i">
 <img alt="BEL" class="fi-BEL fi-flag--4" src="https://api.fifa.com/api/v1/picture/flags-sq-4/bel" title="BEL"/>
 </div>
 <div class="fi-t__n">
 <span class="fi-t__nText">Belgium</span>
 <span class="fi-t__nTri">BEL</span>
 </div>
 </div>
 </a>
 </span></td>
 <td class="fi-table__td fi-table__points"><span class="text">1755</span></td>
 <td class="fi-table__td fi-table__prevpoints"><span class="text">1752</span></td>
 <td class="fi-table__td fi-table__rankingmovement"><span class="text">0</span></td>
 <td class="fi-table__td fi-table__positions"><span class="text"> <svg class="fi-svg-icon fi-module-ranking__ranking__item__arrow"><use xlink:href="#icon-equal"></use></svg

In [127]:
soup = BeautifulSoup(html, 'html.parser')
fifa = soup.select('tbody')[0].find_all('tr')


In [462]:
def rankingFifa(t):
    trr= t.find_all('td')
    try:
        return {
            "Position": trr[0].text,
            "Country": re.sub('\\n', '',trr[1].text.upper().strip())[:-3]
        }
    except:
        return None


In [463]:
rankingfifa = pd.DataFrame(list(filter(lambda x: x ,map(lambda t: rankingFifa(t), scrapping(url)))))

In [464]:
rankingfifa

,Position,Country
0,1,BELGIUM
1,2,FRANCE
2,3,BRAZIL
3,4,ENGLAND
4,5,URUGUAY
...,...,...
205,206,ERITREA
206,206,US VIRGIN ISLANDS
207,208,BRITISH VIRGIN ISLANDS
208,209,SAN MARINO


In [515]:
position1 = list(rankingfifa.loc[(rankingfifa['Country'] == 'BELGIUM'), 'Position'])

In [531]:
position1 = int(position1[0])

In [532]:
type(position1)

int

In [491]:
def web_scrapping(country):
    url = "https://www.fifa.com/fifa-world-ranking/ranking-table/men/"
    def scrapping(url):
        res = requests.get(url)
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')
        result = soup.select('tbody')[0].find_all('tr')
        return result
    
    def rankingFifa(t):
        trr = t.find_all('td')
        try:
            return {
                'Position': trr[0].text,
                'Country': re.sub('\\n', '', trr[1].text.upper().strip())[:-3]
            }
        except:
            return None
        
    rfifa = pd.DataFrame(list(filter(lambda x: x, map(lambda t: rankingFifa(t), scrapping(url)))))
    position = rfifa.loc[rfifa['Country']== country, 'Position'].iloc[0]
    return '{} have the {} position in the current Ranking FIFA'.format(country, position)

In [492]:
web_scrapping('BRAZIL')

'BRAZIL have the 3 position in the current Ranking FIFA'